In [1]:
import matplotlib
matplotlib.use('Agg')

from astropy.io import fits
import glob
import os
import sys
import time
import h5py
import numpy as np
import pandas as pd
import subprocess
from astropy.visualization import ZScaleInterval
from scipy import interpolate

import matplotlib.pyplot as plt 

from joblib import Parallel, delayed

In [2]:
def get_delay(f_l, f_h, DM):
    '''
    Calculate DM delay between two frequencies for given DM
    Args:
        f_l, f_h: float, low/high frequencies in MHz
        DM: float or int, in pc cm^3
    Returns:
        delay due to DM: float, seconds
    '''
    return 4.15*(10.**3)*(f_l**(-2.)-f_h**(-2.))*DM

def get_SB(freq):
    '''
    Return the central frequency of a LOFAR subband, for  LBA.
    Input: MHz (float) 
    Output: Subband (int)
    '''
    return int(np.floor((1024./200.)*freq))

def get_freq(SB):
    '''
    Return the LOFAR subband of a given frequency, for LBA.
    Input: Subband
    Output: MHz (float) 
    '''
    return (SB/512.)*200./2.

def get_beam(freq):
    beams = np.array([30,35,40,50,55,60,65,70,75,80,85,90])
    freq_to_use = str(beams[np.argsort(np.abs(freq-beams))[0]])

    beam_file = "/home/mkuiack1/AARTFAAC_beamsim/LBAOUTER_AARTFAAC_beamshape_{}MHz.hdf5".format(freq_to_use)
    orig =  np.array(h5py.File(beam_file, mode="r").get('lmbeamintensity_norm'))

    # Make its coordinates; x is horizontal.
    x = np.linspace(0, 2300, orig.shape[1])
    y = np.linspace(0, 2300, orig.shape[0])

    # Make the interpolator function.
    f = interpolate.interp2d(x, y, orig, kind='linear')

    # Construct the new coordinate arrays.
    x_new = np.arange(0, 2300)
    y_new = np.arange(0, 2300)

    # Do the interpolation.
    return f(x_new, y_new)

def get_slices(OBS):
    return [os.path.basename(x)[:-4]
            for x in sorted(glob.glob("/zfs/helios/filer?/mkuiack1/{}/*_all"
                               .format(OBS)))]

def get_ddfilelist(filelist, freqs, check_time, DM, integration=1):
    dd_filelist = []
    
    for nth_timestep in range(integration):
        check_time = pd.to_datetime(check_time)+pd.to_timedelta(nth_timestep, unit='s')
        for i in range(len(freqs)):
            try:
                newfile = filelist[str(freqs[i])].loc[pd.to_datetime(check_time)
                                        +pd.to_timedelta(np.round(get_delay(freqs[i],np.max(freqs), DM),0), unit='s') ]
                if str(newfile) == 'nan' :
                    newfile = filelist[str(freqs[i])].loc[pd.to_datetime(check_time)
                          +pd.to_timedelta(np.round(get_delay(freqs[i],np.max(freqs), DM),0), unit='s')-
                         +pd.to_timedelta(1, unit='s')]
                dd_filelist.append(newfile)
            except KeyError:
                try:
                    newfile = filelist[str(freqs[i])].loc[pd.to_datetime(check_time)
                          +pd.to_timedelta(np.round(get_delay(freqs[i],np.max(freqs), DM),0), unit='s')-
                         +pd.to_timedelta(1, unit='s')]
                    dd_filelist.append(newfile)
                except KeyError:
    #                 print "no"
                    continue 
    return dd_filelist

def make_dedisperse_image(dd_filelist):
    
    integrated = np.zeros([1024,1024])
    
    for img in dd_filelist:
        hdu_1 = fits.open(img)[0]
        integrated += hdu_1.data[0,0,:,:]
        
    return integrated

In [3]:
def make_filelist(OBS, SLICE):
    subbands = np.array(sorted([os.path.basename(x)[2:5] for x in  glob.glob("/zfs/helios/filer?/mkuiack1/{}/SB*.vis".format(OBS)) ]), dtype=int)

#     print len(subbands), "subbands in", "/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)
    if len(subbands) == 0:
        print ("/zfs/helios/filer?/mkuiack1/{}/SB*.vis".format(OBS))
        sys.exit()

    filelist = pd.DataFrame([])

    for i in range(len(subbands)):

        if subbands[i] == 256:
            continue 

#         print subbands[i]
        index = pd.to_datetime([os.path.basename(x)[:21]
                                for x in sorted(glob.glob("/zfs/helios/filer?/mkuiack1/{}/{}_all/SB".format(OBS, SLICE)
                                                          +str(subbands[i])
                                                          +"*/imgs/*.fits"))]).round("100ms")

        df = pd.DataFrame({str(get_freq(subbands[i])):\
                           sorted(glob.glob("/zfs/helios/filer?/mkuiack1/{}/{}_all/SB".format(OBS, SLICE)
                                            +str(subbands[i])
                                            +"*/imgs/*.fits"))},
                          index=index)


        filelist = pd.concat([filelist, df.loc[~df.index.duplicated(keep='first')]], axis=1)
    print ("len file list", len(filelist))
    return filelist, index
        
        
        
        

In [4]:
# frame = 0
# for fi_ind in index:

def make_frame(fi_ind):
#    im_stack = np.zeros([2300, 2300])
    f_stack = []
    all_freqs = []
    slice_times = []
    all_std = []
    count = 0 
    psr = []
#     im_stack = []
    im_stack = np.zeros([2300,2300])
    print( fi_ind)
    for img_file in get_ddfilelist(filelist, get_freq(subbands), fi_ind, 0, 
                                   integration=integration):
#             print img_file
            try:
                if os.path.exists("/hddstore/mkuiack1/png/{}.png".format(os.path.basename(img_file)[:21])):
                    print ("/hddstore/mkuiack1/png/{}.png".format(os.path.basename(img_file)[:21])+" done.")
                    return 0 
            except IndexError:
                continue
            except AttributeError:
                continue 
            try:
                data, f = fits.getdata(img_file, header=True)
            except IOError:
                print ("IOError", img_file)
                continue  
            STD = np.std(data[0,0,500:1500,500:1500])


            all_std.append(STD)
    #         print STD
            slice_times.append(f["DATE-OBS"])
            all_freqs.append(f[" CRVAL3"])

            beam_model = get_beam(f[" CRVAL3"]/1e6)
            
            if STD < 30. and STD > 3.:
                f_stack.append(f)
#                 im_stack.append(data[0,0,:,:]/np.abs(np.median(data[0,0,500:1500,500:1500]))
                im_stack += data[0,0,:,:]/np.abs(np.median(data[0,0,500:1500,500:1500]))
                                
    if len(im_stack) == 0:
        print ("no good image in ", fi_ind)
        return 0
    
#     pd.DataFrame(all_std).T.to_csv('img_stds.csv', mode='a', header=False, index=False)
    
    fig = plt.figure(figsize=(10.8, 10.8))
    ax = plt.subplot(111)
    
#     img = np.median(np.array(im_stack), axis=0)
    
    vmin, vmax= ZScaleInterval(contrast=0.4).get_limits(im_stack)
    
    imgmap = plt.imshow(im_stack, origin="lower",
               vmin=vmin, vmax=vmax)
    
    plt.text(.01,.98, f["DATE-OBS"], va="top", ha="left",
             fontsize=16, color="white", transform=ax.transAxes,)
    
    plt.text(.01,.95, "nSB: {}".format(len(f_stack)), va="top", ha="left",
             fontsize=16, color="white", transform=ax.transAxes,)


    imgmap.cmap.set_bad('k',1.)
    ax.patch.set_facecolor('k')

    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

    fig.savefig("/hddstore/mkuiack1/png/{}.png".format(f["DATE-OBS"]), 
                dpi=100, facecolor='black')
    plt.close()
    return 0 

In [5]:
!mkdir /hddstore/mkuiack1/png


for OBS in ["202102230045","202102230145","202102230245"]:
    t1 = time.time()
    subbands = np.array(sorted([os.path.basename(x)[2:5]
                                for x in  glob.glob("/zfs/helios/filer1/mkuiack1/{}/SB*.vis".format(OBS)) ]),
                        dtype=int)

    print (len(subbands), "subbands in", "/zfs/helios/filer1/mkuiack1/{}/SB*.vis".format(OBS))
    if len(subbands) == 0:
        print ("/zfs/helios/filer1/mkuiack1/{}/SB*.vis".format(OBS))
        sys.exit()

    for _slice in get_slices(OBS):
        print (_slice)

        filelist, index = make_filelist(OBS, _slice)
        integration = 1
        with Parallel(n_jobs=9) as parallel:
            _out = parallel(delayed(make_frame)(fi_ind) 
                                      for fi_ind in index[::integration])

        subprocess.call(["rsync", "--remove-source-files",
                     "-av", "/hddstore/mkuiack1/png",
                     "/zfs/helios/filer1/mkuiack1/{}".format(OBS)])

    print ("Done.", OBS, round(time.time()-t1, 2))

mkdir: cannot create directory ‘/hddstore/mkuiack1/png’: File exists
16 subbands in /zfs/helios/filer1/mkuiack1/202102230045/SB*.vis
2021-02-23T00:50:13-00:53:23
len file list 189
2021-02-23T00:53:13-00:56:23
len file list 189
2021-02-23T00:56:13-00:59:23
len file list 188
2021-02-23T00:59:13-01:02:23
len file list 189
2021-02-23T01:02:13-01:05:23
len file list 189
2021-02-23T01:05:13-01:08:23
len file list 189
2021-02-23T01:08:13-01:11:23
len file list 189
2021-02-23T01:11:13-01:14:23
len file list 188
2021-02-23T01:14:13-01:17:23
len file list 189
2021-02-23T01:17:13-01:20:23
len file list 189
2021-02-23T01:20:13-01:23:23
len file list 189
2021-02-23T01:23:13-01:26:23
len file list 189
2021-02-23T01:26:13-01:29:23
len file list 188
2021-02-23T01:29:13-01:32:23
len file list 188
2021-02-23T01:32:13-01:35:23
len file list 189
2021-02-23T01:35:13-01:38:23
len file list 189
2021-02-23T01:38:13-01:41:23
len file list 189
2021-02-23T01:41:13-01:44:23
len file list 189
2021-02-23T01:44:13-0

In [61]:
filelist, index = make_filelist(OBS, "2021-02-22T20:48:37-20:51:47")
for fi_ind in index:
    print(fi_ind)

len file list 189
2021-02-22 20:48:36.500000
2021-02-22 20:48:37.500000
2021-02-22 20:48:38.500000
2021-02-22 20:48:39.500000
2021-02-22 20:48:40.500000
2021-02-22 20:48:41.500000
2021-02-22 20:48:42.500000
2021-02-22 20:48:43.500000
2021-02-22 20:48:44.500000
2021-02-22 20:48:45.500000
2021-02-22 20:48:46.500000
2021-02-22 20:48:47.500000
2021-02-22 20:48:48.600000
2021-02-22 20:48:49.600000
2021-02-22 20:48:50.600000
2021-02-22 20:48:51.600000
2021-02-22 20:48:52.600000
2021-02-22 20:48:53.600000
2021-02-22 20:48:54.600000
2021-02-22 20:48:55.600000
2021-02-22 20:48:56.600000
2021-02-22 20:48:57.600000
2021-02-22 20:48:58.600000
2021-02-22 20:48:59.600000
2021-02-22 20:49:00.600000
2021-02-22 20:49:01.600000
2021-02-22 20:49:02.600000
2021-02-22 20:49:03.700000
2021-02-22 20:49:04.700000
2021-02-22 20:49:05.700000
2021-02-22 20:49:06.700000
2021-02-22 20:49:07.700000
2021-02-22 20:49:08.700000
2021-02-22 20:49:09.700000
2021-02-22 20:49:10.700000
2021-02-22 20:49:11.700000
2021-02-22

In [13]:
SLICE = "2021-02-22T21:06:37-21:09:47"
OBS = "202102222045"

In [14]:
filelist, index = make_filelist(OBS, SLICE)

len file list 182


In [24]:
filelist.keys().astype("float")

Float64Index([ 48.828125, 51.3671875, 52.1484375, 53.3203125,  54.296875,
                55.46875,   57.03125,    57.8125, 58.3984375,  60.546875,
               62.109375,  62.890625, 63.8671875,  64.453125, 65.4296875,
                69.53125],
             dtype='float64')

In [28]:
len(get_ddfilelist(filelist, filelist.keys().astype("float"), index[100], 0, integration=10))

160

In [15]:
# OBS = 202012131700#sys.argv[1] #"202012032122"

subbands = np.array(sorted([os.path.basename(x)[2:5] for x in  glob.glob("/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)) ]), dtype=int)

print len(subbands), "subbands in", "/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)
if len(subbands) == 0:
    print "/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)
    sys.exit()


# subbands = np.array([281, 284, 287, 291, 294, 298, 301, 304, 
#                      308, 311, 315, 318, 321, 325, 328, 332])

# subbands = np.array([156, 165, 174, 187, 195, 213, 221, 231,
#                      243, 256, 257, 267, 278, 284, 296, 320])

# OBS = "202012032122"

16 subbands in /zfs/helios/filer0/mkuiack1/202012131700/SB*.vis


In [25]:
# SLICE = "2020-12-13T17:02:01-17:05:11"

filelist = pd.DataFrame([])

for i in range(len(subbands)):
    
    if subbands[i] == 256:
        continue 

    print subbands[i]
    index = pd.to_datetime([os.path.basename(x)[:21]
                            for x in sorted(glob.glob("/zfs/helios/filer0/mkuiack1/{}/{}_all/SB".format(OBS, SLICE)
                                                      +str(subbands[i])
                                                      +"*/imgs/*.fits"))]).round("1s")
    
    df = pd.DataFrame({str(get_freq(subbands[i])):\
                       sorted(glob.glob("/zfs/helios/filer0/mkuiack1/{}/{}_all/SB".format(OBS, SLICE)
                                        +str(subbands[i])
                                        +"*/imgs/*.fits"))},
                      index=index)
    
    
    filelist = pd.concat([filelist, df.loc[~df.index.duplicated(keep='first')]], axis=1)

156
165
174
187
195
213
221
231
243
250
257
267
278
284
296
320


In [5]:
SLICE = "2020-12-14T02:02:01-02:05:11*"
OBS = "202012140200"

filelist, index = make_filelist(OBS, SLICE)

len file list 182


In [37]:

_out = Parallel(n_jobs=6)(delayed(make_frame)(fi_ind) for fi_ind in index)

mkdir: cannot create directory ‘/hddstore/mkuiack1/png’: File exists


In [38]:
subprocess.call(["rsync", "--remove-source-files", 
                 "-av", "/hddstore/mkuiack1/png", 
                 "/zfs/helios/filer0/mkuiack1/{}/png".format(OBS)])

0

In [28]:
pd.DataFrame(np.array([0.1,0.3,0.5]).T,columns=[1,2,3], index=pd.to_datetime("2020-12-12 20:20:20"))

TypeError: Index(...) must be called with a collection of some kind, Timestamp('2020-12-12 20:20:20') was passed

In [7]:
 subbands

array([156, 165, 174, 187, 195, 213, 221, 231, 243, 250, 257, 267, 278,
       284, 296, 320])

In [ ]:
!mkdir /hddstore/mkuiack1/png

# "2020-12-13T17:17:01-17:20:11", 
#                "2020-12-13T17:20:01-17:23:11",
#                "2020-12-13T17:23:01-17:26:11",
#                "20b20-12-13T17:26:01-17:29:11",
#                "2020-12-13T17:29:01-17:32:11"
# SLICE = "2020-12-13T17:14:01-17:17:11"

for OBS in ["202101031339"]:
    subbands = np.array(sorted([os.path.basename(x)[2:5] 
                                for x in  glob.glob("/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)) ]),
                        dtype=int)

    if len(subbands) == 0:
        print "/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)
        sys.exit()

    for _slice in [
"2021-01-03T13:47:01-13:50:11",
"2021-01-03T13:50:01-13:53:11",
"2021-01-03T13:53:01-13:56:11",
"2021-01-03T13:56:01-13:59:11",
"2021-01-03T13:59:01-14:02:11",
"2021-01-03T14:02:01-14:05:11",
"2021-01-03T14:05:01-14:08:11",
"2021-01-03T14:08:01-14:11:11",
"2021-01-03T14:11:01-14:14:11",
"2021-01-03T14:14:01-14:17:11",
"2021-01-03T14:17:01-14:20:11",
"2021-01-03T14:20:01-14:23:11",
"2021-01-03T14:23:01-14:26:11",
"2021-01-03T14:26:01-14:29:11",
"2021-01-03T14:29:01-14:32:11",
"2021-01-03T14:32:01-14:35:11"]:
        print _slice
        filelist, index = make_filelist(OBS, _slice)

        _out = Parallel(n_jobs=6)(delayed(make_frame)(fi_ind) for fi_ind in index)

        subprocess.call(["rsync", "--remove-source-files",
                         "-av", "/hddstore/mkuiack1/png",
                         "/zfs/helios/filer0/mkuiack1/{}/png".format(OBS)])

    print "Done."

mkdir: cannot create directory ‘/hddstore/mkuiack1/png’: File exists
2021-01-03T13:47:01-13:50:11
len file list 174


In [ ]:
["2021-01-03T13:41:01-13:44:11",
"2021-01-03T13:44:01-13:47:11",
"2021-01-03T13:47:01-13:50:11",
"2021-01-03T13:50:01-13:53:11",
"2021-01-03T13:53:01-13:56:11",
"2021-01-03T13:56:01-13:59:11",
"2021-01-03T13:59:01-14:02:11",
"2021-01-03T14:02:01-14:05:11",
"2021-01-03T14:05:01-14:08:11",
"2021-01-03T14:08:01-14:11:11",
"2021-01-03T14:11:01-14:14:11",
"2021-01-03T14:14:01-14:17:11",
"2021-01-03T14:17:01-14:20:11",
"2021-01-03T14:20:01-14:23:11",
"2021-01-03T14:23:01-14:26:11",
"2021-01-03T14:26:01-14:29:11",
"2021-01-03T14:29:01-14:32:11",
"2021-01-03T14:32:01-14:35:11"]

In [ ]:
!mkdir /hddstore/mkuiack1/png
import time 


for OBS in ["202101040500","202101040400","202101040300","202101032000","202101040000"]:
    t1 = time.time()
    subbands = np.array(sorted([os.path.basename(x)[2:5] 
                                for x in  glob.glob("/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)) ]),
                        dtype=int)

    print len(subbands), "subbands in", "/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)
    if len(subbands) == 0:
        print "/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)
        sys.exit()



    for _slice in ["2021-01-??T??:02:01-??:05:11",
                   "2021-01-??T??:05:01-??:08:11",
                   "2021-01-??T??:08:01-??:11:11",
                   "2021-01-??T??:11:01-??:14:11",
                   "2021-01-??T??:14:01-??:17:11",
                   "2021-01-??T??:17:01-??:20:11",
                   "2021-01-??T??:20:01-??:23:11",
                   "2021-01-??T??:23:01-??:26:11",
                   "2021-01-??T??:26:01-??:29:11",
                   "2021-01-??T??:29:01-??:32:11",
                   "2021-01-??T??:32:01-??:35:11",
                   "2021-01-??T??:35:01-??:38:11",
                   "2021-01-??T??:38:01-??:41:11",
                   "2021-01-??T??:41:01-??:44:11",
                   "2021-01-??T??:44:01-??:47:11",
                   "2021-01-??T??:47:01-??:50:11",
                   "2021-01-??T??:50:01-??:53:11",
                   "2021-01-??T??:53:01-??:56:11"]:
        print _slice

        filelist, index = make_filelist(OBS, _slice)

        with Parallel(n_jobs=9) as parallel:
            _out = parallel(delayed(make_frame)(fi_ind)
                                      for fi_ind in index)

        subprocess.call(["rsync", "--remove-source-files",
                     "-av", "/hddstore/mkuiack1/png",
                     "/zfs/helios/filer0/mkuiack1/{}/png".format(OBS)])

    print "Done.", OBS, round(t1-time.time(), 2)

mkdir: cannot create directory ‘/hddstore/mkuiack1/png’: File exists
16 subbands in /zfs/helios/filer0/mkuiack1/202101040500/SB*.vis
2021-01-??T??:02:01-??:05:11
len file list 182
2021-01-??T??:05:01-??:08:11
len file list 182
2021-01-??T??:08:01-??:11:11
len file list 174


In [7]:
    for _slice in ["2021-01-??T??:02:01-??:05:11",
                   "2021-01-??T??:05:01-??:08:11",
                   "2021-01-??T??:08:01-??:11:11",
                   "2021-01-??T??:11:01-??:14:11",
                   "2021-01-??T??:14:01-??:17:11",
                   "2021-01-??T??:17:01-??:20:11",
                   "2021-01-??T??:20:01-??:23:11",
                   "2021-01-??T??:23:01-??:26:11",
                   "2021-01-??T??:26:01-??:29:11",
                   "2021-01-??T??:29:01-??:32:11",
                   "2021-01-??T??:32:01-??:35:11",
                   "2021-01-??T??:35:01-??:38:11",
                   "2021-01-??T??:38:01-??:41:11",
                   "2021-01-??T??:41:01-??:44:11",
                   "2021-01-??T??:44:01-??:47:11",
                   "2021-01-??T??:47:01-??:50:11",
                   "2021-01-??T??:50:01-??:53:11",
                   "2021-01-??T??:53:01-??:56:11"]:

["2020-12-1?T??:02:01-??:05:11",
                   "2020-12-1?T??:05:01-??:08:11",
                   "2020-12-1?T??:08:01-??:11:11",
                   "2020-12-1?T??:14:01-??:17:11",
                   "2020-12-1?T??:17:01-??:20:11",
                   "2020-12-1?T??:20:01-??:23:11",
                   "2020-12-1?T??:23:01-??:26:11",
                   "2020-12-1?T??:26:01-??:29:11",
                   "2020-12-1?T??:29:01-??:32:11",
                   "2020-12-1?T??:32:01-??:35:11",
                   "2020-12-1?T??:35:01-??:38:11",
                   "2020-12-1?T??:38:01-??:41:11",
                   "2020-12-1?T??:41:01-??:44:11",
                   "2020-12-1?T??:44:01-??:47:11",
                   "2020-12-1?T??:47:01-??:50:11",
                   "2020-12-1?T??:50:01-??:53:11",
                   "2020-12-1?T??:53:01-??:56:11"]

OBS = "202012032122"

"2020-12-04T00:30:39-00:33:49",
"2020-12-04T00:33:39-00:36:49",
"2020-12-04T00:36:39-00:39:49",
"2020-12-04T00:39:39-00:42:49",
"2020-12-04T00:42:39-00:45:49",
"2020-12-04T00:45:39-00:48:49",
"2020-12-04T00:48:39-00:51:49",
"2020-12-04T00:51:39-00:54:49",
"2020-12-04T00:54:39-00:57:49",
"2020-12-04T00:57:39-01:00:49",
"2020-12-04T01:00:39-01:03:49",
"2020-12-04T01:03:39-01:06:49",
"2020-12-04T01:06:39-01:09:49",
"2020-12-04T01:09:39-01:12:49",
"2020-12-04T01:12:39-01:15:49",
"2020-12-04T01:15:39-01:18:49",
"2020-12-04T01:18:39-01:21:49",
"2020-12-04T01:21:39-01:24:49",
"2020-12-04T01:24:39-01:27:49",
"2020-12-04T01:27:39-01:30:49",
"2020-12-04T01:30:39-01:33:49",
"2020-12-04T01:33:39-01:36:49",
"2020-12-04T01:36:39-01:39:49",
"2020-12-04T01:39:39-01:42:49",
"2020-12-04T01:42:39-01:45:49",
"2020-12-04T01:45:39-01:48:49",
"2020-12-04T01:48:39-01:51:49",
"2020-12-04T01:51:39-01:54:49",
"2020-12-04T01:54:39-01:57:49",
"2020-12-04T01:57:39-02:00:49",
"2020-12-04T02:00:39-02:03:49",
"2020-12-04T02:03:39-02:06:49",
"2020-12-04T02:06:39-02:09:49",
"2020-12-04T02:09:39-02:12:49",
"2020-12-04T02:12:39-02:15:49",
"2020-12-04T02:15:39-02:18:49",
"2020-12-04T02:27:39-02:30:49",
"2020-12-04T02:18:39-02:21:49",
"2020-12-04T02:21:39-02:24:49",
"2020-12-04T02:30:39-02:33:49",
"2020-12-04T02:24:39-02:27:49",



In [7]:
subbands = np.array(sorted([os.path.basename(x)[2:5] 
                            for x in  glob.glob("/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)) ]),
                    dtype=int)


make_frame(pd.to_datetime('2020-12-14 02:04:49'))

2020-12-14 02:04:49


/home/mkuiack1/env/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide
/home/mkuiack1/env/lib/python2.7/site-packages/numpy/core/_methods.py:75: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


0

In [27]:
# OBS = "202012140200"
# # OBS = "202012032122"
# subbands = np.array(sorted([os.path.basename(x)[2:5] 
#                             for x in  glob.glob("/zfs/helios/filer0/mkuiack1/{}/SB*.vis".format(OBS)) ]),
#                     dtype=int)
# SLICE = "2020-12-14T02:02:01-02:05:11*"

image = make_frame(pd.to_datetime('2020-12-14 02:03:49'))



2020-12-14 02:03:49


/home/mkuiack1/env/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide


In [28]:
np.shape(image)

(2300, 2300)

In [29]:
# import matplotlib.image as mpimg
# import matplotlib.pyplot as plt

# matplotlib.use('TkAgg')

# Read Images 
plt.figure(figsize=(10,10))
# img = mpimg.imread('/hddstore/mkuiack1/png/2020-12-13T17:33:25.1.png') 
  
# Output Images 
plt.imshow(image) 


In [40]:
SLICE = "2020-12-13T21:1*"
OBS = "202012132100"

filelist, index = make_filelist(OBS, SLICE)

len file list 524


In [37]:
get_ddfilelist(filelist, get_freq(subbands), fi_ind, 0)

['/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB156-2020-12-13T21:08:01-21:11:11/imgs/2020-12-13T21:10:37.4-SB156.fits',
 '/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB165-2020-12-13T21:08:01-21:11:11/imgs/2020-12-13T21:10:37.4-SB165.fits',
 '/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB174-2020-12-13T21:08:01-21:11:11/imgs/2020-12-13T21:10:37.4-SB174.fits',
 '/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB187-2020-12-13T21:08:01-21:11:11/imgs/2020-12-13T21:10:37.4-SB187.fits',
 '/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB195-2020-12-13T21:08:01-21:11:11/imgs/2020-12-13T21:10:37.4-SB195.fits',
 '/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB213-2020-12-13T21:08:01-21:11:11/imgs/2020-12-13T21:10:37.4-SB213.fits',
 '/zfs/helios/filer0/mkuiack1/202012132100/2020-12-13T21:08:01-21:11:11_all/SB221-2020-12-13T2

In [8]:
fi_ind = '2020-12-14 02:04:49'

im_stack = np.zeros([2300, 2300])
all_freqs = []
slice_times = []
all_std = []
count = 0 
im_stack = []

for img_file in get_ddfilelist(filelist, get_freq(subbands), fi_ind, 0):

        try:
            data, f=fits.getdata(img_file, header=True)
        except IOError:
            print "IOError", img_file
            continue

        STD = np.std(data[0,0,:,:])
        
        slice_times.append(f["DATE-OBS"])
        all_freqs.append(f[" CRVAL3"]/1e6)

        beam_model = get_beam(f[" CRVAL3"]/1e6)

        if STD < 80. and STD > 3.:
            im_stack.append(data[0,0,:,:]/np.abs(np.median(data[0,0,500:1500,500:1500]))*(np.max(beam_model)/beam_model))

if len(im_stack) == 0:
    print "no good image in ", fi_ind


# hdul = fits.open(img_file)
# prihdr = hdul[0].header

# prihdr.update({'RESTFRQ':np.mean(all_freqs)*1e6})
# prihdr.update({'RESTFREQ':np.mean(all_freqs)*1e6})
# prihdr.update({'RESTBW':hdul[0].header['CDELT3']*len(all_freqs)})
# prihdr.update({'BANDS':",".join(np.array([round(x,2) for x in all_freqs], dtype=str))})
# prihdr.update({'DATE-OBS':min(slice_times)})

# hdul[0].data[0,0,:,:] = np.median(np.array(im_stack), axis=0)
# print "/home/mkuiack1/{}_nSB{}.fits".format(slice_times[0],int(len(all_freqs)))
# hdul.writeto("/home/mkuiack1/{}_nSB{}.fits".format(slice_times[0],int(len(all_freqs)))) 
# print "done."

/home/mkuiack1/env/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in divide


In [17]:
",".join(np.array([round(x,2) for x in [56.2342,34.2342,23.34345,23.3245,23.234]], dtype=str))

'56.23,34.23,23.34,23.32,23.23'

In [11]:
plt.figure()
plt.imshow(np.median(np.array(im_stack), axis=0)) 
plt.show()

In [7]:
basedir = "/zfs/helios/filer0/mkuiack1/202101032100"

img_files = glob.glob(basedir+"/2021-01-03T21:02:01-21:05:11_all/SB*/imgs/2021-01-03T21:02:31*")
print len(img_files)

16


In [8]:

all_freqs = []
slice_times = []
all_std = []
count = 0 
im_stack = []

for img_file in img_files:

        try:
            data, f=fits.getdata(img_file, header=True)
        except IOError:
            print "IOError", img_file
            continue

        STD = np.std(data[0,0,:,:])
        
        slice_times.append(f["DATE-OBS"])
        all_freqs.append(f[" CRVAL3"]/1e6)

        beam_model = get_beam(f[" CRVAL3"]/1e6)

#         if STD < 80. and STD > 3.:
        im_stack.append(data[0,0,:,:]/np.abs(np.median(data[0,0,500:1500,500:1500]))*(np.max(beam_model)/beam_model))

    

/home/mkuiack1/env/lib/python2.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in divide


In [11]:
fig = plt.figure(figsize=(10.8, 10.8))
ax = plt.subplot(111)

img = np.median(np.array(im_stack), axis=0)

vmin, vmax= ZScaleInterval(contrast=0.4).get_limits(img)

imgmap = plt.imshow(img, origin="lower",
           vmin=vmin, vmax=vmax)

plt.text(.01,.98, f["DATE-OBS"], va="top", ha="left",
         fontsize=16, color="white", transform=ax.transAxes,)

plt.text(.01,.95, "nSB: {}".format(len(all_freqs)), va="top", ha="left",
         fontsize=16, color="white", transform=ax.transAxes,)


imgmap.cmap.set_bad('k',1.)
ax.patch.set_facecolor('k')

plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()